# PDFの全ページを画像（jpg or png）に変換

In [ ]:
import os
from pathlib import Path
from pdf2image import convert_from_path
poppler_path =  r"C:\soft\poppler-22.04.0\Library\bin"

def pdf_to_image(file_name, ext="png", param_dpi=400): #PDF前ページをimgに変換
    #param_dpiは出力解像度（デフォルトは200だが関数内では上記数値をデフォルトとした）
    pdf_path = Path(file_name) # PDFファイルのパス
    img_path_base = ""
    (name, extention) = os.path.splitext(file_name)
    images = convert_from_path(pdf_path ,poppler_path=poppler_path, dpi=param_dpi) 
    for i in range(len(images)):
        if ext == "jpg":
            images[i].save(img_path_base + name + '[%idpi]-'%(param_dpi) + str(i+1)+'.jpg', 'JPEG')
        elif ext == "png":
            images[i].save(img_path_base + name + '[%idpi]-'%(param_dpi) + str(i+1)+'.png', 'PNG')
        else:
            print("EXTENTION ERROR")

#ここから
pdf_name = "original.pdf"
pdf_to_image(pdf_name,ext="png",param_dpi=400) #ファイル指定

# 見開きスキャンPDFを左右2ページのPDFに変換

In [ ]:
#https://www.robotech-note.com/entry/2018/04/28/%E4%B8%AD%E5%BF%83%E3%81%AE%E3%81%9A%E3%82%8C%E3%81%9F%E8%A6%8B%E9%96%8B%E3%81%8D%E3%82%B9%E3%82%AD%E3%83%A3%E3%83%B3PDF%E3%82%92%EF%BC%92%E3%83%9A%E3%83%BC%E3%82%B8%E3%81%AB%E5%88%86%E5%89%B2%E3%81%99

from PyPDF2 import PdfFileWriter, PdfFileReader

def pdf_RLdivider2(filename):
    output_filename = "output.pdf"

    original_L = PdfFileReader(filename)
    original_R = PdfFileReader(filename)

    marged = PdfFileWriter()

    page_num = original_L.getNumPages()

    # 各方面の縁を削除する
    # 数値をプラスすると、mm単位で縁の余白を切り取ることができる
    bottom = 10
    top = 10
    right = 20
    left = 25

    # 左ページのセンター余白、プラスすると左ページが面積が減る(mm)
    center_left = -5
    # 右ページのセンター余白、プラスすると右ページが面積が減る(mm)
    center_right = 10

    for i in range(0, page_num):
        # crop LEFT
        page_L = original_L.getPage(i)
        page_L.mediaBox.upperRight = (
            page_L.mediaBox.getUpperRight_x() / 2 - center_left,
            page_L.mediaBox.getUpperRight_y()-top
        )
        page_L.mediaBox.lowerLeft = (
            page_L.mediaBox.getLowerLeft_x()+left,
            page_L.mediaBox.getLowerLeft_y()+bottom
        )
        marged.addPage(page_L)


        # crop RIGHT
        page_R = original_R.getPage(i)
        page_R.mediaBox.upperRight = (
            page_R.mediaBox.getUpperRight_x()-right,
            page_R.mediaBox.getUpperRight_y()-top
        )
        page_R.mediaBox.lowerLeft = (
            page_R.mediaBox.getUpperRight_x()/2 + center_right,
            page_R.mediaBox.getLowerLeft_y()+bottom
        )
        marged.addPage(page_R)

    with open(output_filename, mode="wb") as f:
        marged.write(f)

#ここから
pdf1 = "original.pdf"
pdf_RLdivider2(pdf1)

# PDFのパスワードを解除して保存

In [ ]:
from pikepdf import Pdf

def pdf_del_password(password, filename, newfilename):
    pdf = Pdf.open(filename, password=password)
    pdf_unlock = Pdf.new()
    pdf_unlock.pages.extend(pdf.pages)
    pdf_unlock.save(newfilename)

# ここから
password = 'password'   #pdfのパスワード
filename = "locked.pdf"   #パスワードを解除したいpdfの名前
newfilename = "unlocked.pdf"   #パスワード解除後のpdfの名前
pdf_del_password(password, filename, newfilename)

# PDFを白黒変換

In [ ]:
from pdf2image import convert_from_path
import img2pdf
import glob
import os
import cv2
poppler_path =  r"C:\soft\poppler-22.04.0\Library\bin"

def my_mkdir(s):
    si = s.split("/")[-1]
    try:
        os.mkdir(s)
    except:
        print("mkdir ERROR %s"%(si))

def pdf_to_BlackWhitePdf(pdf_name):
    # 各フォルダの作成
    pdf_name0 = pdf_name.split(".")[0]
    thresh = 170
    my_mkdir("./content/img_dir")
    my_mkdir("./content/img_dir/"+pdf_name0)
    my_mkdir("./content/result_dir")
    # 各フォルダのパス
    pdf_path = "./content/pdf_file/"+pdf_name
    img_dir = "./content/img_dir/"+pdf_name0+"/"
    result_path = "./content/result_dir/"+pdf_name
    # PDFをImage に変換（150dpi）
    pages = convert_from_path(str(pdf_path), 150, poppler_path=poppler_path)
    # 画像ファイルを１ページずつ保存
    for i, page in enumerate(pages):
        file_name = img_dir + "{:02d}".format(i + 1) + ".jpeg"
        img_path = file_name
        # JPEGで保存
        page.save(str(img_path), "JPEG")
        src=cv2.imread(str(img_path), 0)
        ret, dst = cv2.threshold(src, thresh, 255, cv2.THRESH_BINARY)
        cv2.imwrite(str(img_path), dst)
    #pdfに変換
    extension  = ".jpeg"
    img_list=glob.glob(img_dir+"*"+extension)
    img_list = [im.replace('\\', '/') for im in img_list] #パス変換
    print(img_list)
    with open(result_path,"wb") as f:
        f.write(img2pdf.convert(sorted(img_list),title=pdf_name, viewer_page_layout=img2pdf.parse_layout('onecolumn')))

#ここから
pdf1 = "original.pdf"
pdf_to_BlackWhitePdf()

# PDFを分解する 

In [ ]:
#Pythonで簡単にPDFを分割する
#https://reffect.co.jp/python/python-divide-pdf-file
import glob,os #(1)
from PyPDF2 import PdfFileWriter, PdfFileReader #(2)

def pdf_divide(file_name):   
    (name, extention) = os.path.splitext(file_name) #(4)
    pdf_file_reader = PdfFileReader(file_name) #(5)
    page_nums = pdf_file_reader.getNumPages() #(6)
    for num in range(page_nums): #(7)
        file_object = pdf_file_reader.getPage(num) #(7)
        pdf_file_name = name + '-' + str(num+1) + '.pdf' #(8)
        pdf_file_writer = PdfFileWriter() #(9)
        with open(pdf_file_name,'wb') as f: #(10)
            pdf_file_writer.addPage(file_object) #(11)
            pdf_file_writer.write(f) #(12)

#ここから
pdf1 = "original.pdf"
pdf_divide(pdf1)

# PDF(A)の途中ページにPDF(B)挿入

In [ ]:
from PyPDF2 import PdfFileReader, PdfFileWriter
import csv
import os,sys

def insert(pdf1, pdf2, npage):#挿入 元のページに挿入する場合 #指定PDF1のn+1ページ以降に指定PDF2の全ページを挿入する。
    # PdfFileWriterを作成
    writer = PdfFileWriter()
    save = "combine3page.pdf"

    # 元のファイルを開く
    reader = PdfFileReader(pdf1)
    pages = reader.pages
    reader2 = PdfFileReader(pdf2)
    pages2 = reader2.pages
    print(npage)
    
    for i in range(len(pages)):
        page = pages[i]
        if i == npage-1:
            writer.addPage(page) # PdfFileWriterに抽出したページを追加する
            for page2 in pages2:
                writer.addPage(page2)
        else:
            writer.addPage(page)

    # 1ページ目を結合したPDFを書き出す
    with open(save, "wb") as f:
        writer.write(f)
#ここから
pdfA = "003.pdf"
pdfB = "001.pdf"
npage = 10
insert(pdfA, pdfB, npage)

# 複数PDFの複数ページをPDF(A)の指定ページに挿入

In [ ]:
def multi_insert(csv_in = "./multiinsert.csv", ):#複数ファイルからの挿入 元のページに挿入する場合 #指定PDF1のn+1ページ以降に指定PDF2の全ページを挿入する。
    
    
    writer = PdfFileWriter()
    reader = PdfFileReader(pdf1)
    pages = reader.pages
    
    with open(csv_in, encoding="utf-8") as f: #csvの読込み
        reader = csv.reader(f, delimiter=',')
        l = [row for row in reader]
    
    keys = [] #開くpdfファイルのユニーク化
    ip = [] #1列目（元PDFのページ番号だけリスト化）
    for li in l:
        ip.append(int(li[0])) #元PDFのページ番号指定
        if not li[1] in keys:
            keys.append(li[1])

    #辞書のキー毎にインスタンス作成して保管
    dict2 = {}
    for ki in keys:
        reader2 = PdfFileReader(ki)
        pages2 = reader2.pages
        dict2[ki] = pages2

    #元PDFのページ毎に書き込み
    for i in range(len(pages)):
        i1 = i+1 #ページ番号変換
        #print(i, indlist0)
        page = pages[i]
        writer.addPage(page) #元PDFそのままの追加
        if i1 in ip:
            insl = l[ip.index(i1)]
            insi = list(range(int(insl[2]), int(insl[3])+1)) #挿入元のPDF番号リスト
            print(insi)
            insd = dict2[insl[1]] #挿入元のPDFインスタンス呼び出し
            for j in insi:
                writer.addPage(insd[j-1]) #indページ番号で1ずれを修正

    # 1ページ目を結合したPDFを書き出す
    with open(save, "wb") as f:
        writer.write(f)


pdf1 = "original.pdf"
pdf_new = "multiinsert.pdf"
multi_insert(pdf1, pdf_new, csv_in = "./multiinsert.csv") #複数挿入　複数PDFの複数ページを指定ページに挿入する。   

# 複数ファイルの複数ページをPDF(A)と入替

In [ ]:
def multi_deinsert(pdf1, csv_in):#入替 元の数ページを抜いて挿入する場合（1回だけ）
    # PdfFileWriterを作成
    writer = PdfFileWriter()
    base_file_pdf = os.path.splitext(os.path.basename(pdf1))[0] #拡張子なしのファイル名
    #pdf2 = "20221105191350.pdf"
    save = base_file_pdf + "_deinsert.pdf"
    with open(csv_in, encoding="utf-8") as f: #csvの読込み
        reader = csv.reader(f, delimiter=',')
        l = [row for row in reader] #csvに記載のファイルとページ番号のリスト（str）
    #print(l)
    
    #npage = 16 #元PDFの該当ページ範囲は除外
    #npage2 = 17 #元PDFの該当ページ範囲は除外
    
    # 元のファイルを開く
    reader = PdfFileReader(pdf1)
    pages = reader.pages
    
    j = 0 #初期値（今回フラグ）
    j_pre = 0#初期値（前回フラグ）
    for i in range(len(pages)): #ページループ
        page = pages[i]
        j_pre = j
        for k in range(len(l)): #csvファイルのcolmnループ
            npage = int(l[k][1])
            npage2 = int(l[k][2])
            pdf2 = l[k][0]

            if npage <= i+1 and i+1 <= npage2 :
                if j == 0: #フラグを立てる最初の一撃
                    reader_2 = PdfFileReader(pdf2) #なぜかこの場所に配置しないとそれ以降のページが空白に。
                    pages_2 = reader_2.pages
                    for page_2 in pages_2:
                        writer.addPage(page_2)
                    #l.pop(k) 
                    #j_pre = j
                    j = 1 #フラグを立てる。
                    break #kループを出る。

                elif j == 1:
                    if i+1 == npage2: #ページ範囲の最後の時、jを0に戻す。
                        l.pop(k) ##lリストのk番目の要素を削除して2回目の挿入を防ぐ
                        #j_pre = j
                        j = 0 #フラグを戻す
                        break

        if j_pre == 0 and j == 0: #前回jフラグと今回jフラグが0の時実行
            writer.addPage(pages[i])

        print(i+1, j,j_pre, l,npage,i+1,npage2)

    # 1ページ目を結合したPDFを書き出す
    with open(save, "wb") as f:
        writer.write(f)

csv_in = "./toc_mdeinsert.csv"
pdf1 = "original.pdf"
multi_deinsert(pdf1, csv_in)

# PDFのページ抽出（CSV指定）

In [ ]:
#ページ番号指定する場合(csvのパス)
def csv_to_index(csv_in = "./toc.csv"):
    with open(csv_in) as f: #csvの読込み
        reader = csv.reader(f, delimiter=',')
        l = [row for row in reader]
    indlist = []
    for li in l:
        if li[0] == li[1]:
            indlist.append(int(li[0]))
        else:
            indlist += range(int(li[0]), int(li[1])+1)
    return(indlist) #csvから指定されるインデックス番号文字列

def extract(pdf1): #抽出
    base_file_pdf = os.path.splitext(os.path.basename(pdf1))[0] #拡張子なしのファイル名
    save = base_file_pdf + "_目次.pdf" #拡張子なしのファイル名
    #save = "[SABTEC] 機械式定着工法デザインマニュアル(2014)_目次.pdf"
    indfile = base_file_pdf + "_ind.csv " #拡張子なしのファイル名

    addlist = csv_to_index(csv_in = indfile) #csvをリストに変換（エラー修正　中の数値がない問題）
    writer = PdfFileWriter() # PdfFileWriterを作成
    reader = PdfFileReader(pdf1)
    pages = reader.pages
    
    for i in range(len(pages)):
        if i+1 in addlist: #ページ番号リストに含まれるものだけ抽出
            writer.addPage(pages[i]) # PdfFileWriterに抽出したページを追加する
            
    with open(save, "wb") as f:
        writer.write(f)

pdf1 = "original.pdf"
extract(pdf1) #抽出 1つのPDFファイルについてcsvの指定ページ以外除去

# PDFを合体させる（フォルダ+CSV指定）

In [ ]:
from PyPDF2 import PdfFileMerger
import os
import csv
import numpy as np

#ディレクトリないのpdfを名前順にマージする。
def dir_filelist(s, dirname="./"):
    flist = []
    for file in os.listdir(dirname):
        base, ext = os.path.splitext(file)
        if ext == s:
            flist.append(file)
    sorted(flist) #名前の順に並べ替え（する前から順になってるが一応）
    if dirname == "./":
        None
    else:
        flist2 = [] #cdからの相対参照に修正する。
        for fi in flist:
            flist2.append(dirname+"/"+fi)
        flist = flist2
    return(flist)

def get_finds(s, flist):
    i = 0
    for fi in flist:
        if s == fi.split(".")[0]:
            return(i)
        i += 1

#合致するファイルを指定する場合(ファイル名, csvのパス)
def csv_to_merge(flist, csv_in = "./toc.csv"):
    with open(csv_in) as f: #csvの読込み
        reader = csv.reader(f, delimiter=',')
        l = [row for row in reader]
    gatchlist = []
    for li in l:
        if li[0] == li[1]:
            n = get_finds(li[0], flist)
            gatchlist.append(n)
        else:
            n1 = get_finds(li[0], flist)
            n2 = get_finds(li[1], flist)
            nlist = list(np.arange(n1, n2+1))
            gatchlist += nlist
    return(gatchlist) #csvから指定されるファイルのインデックス番号文字列

#指定インデックス以外のファイル除去
def kill_unindfile(flist, gatchlist):
    newflist = []
    for i in range(len(flist)):
        if i in gatchlist:
            newflist.append(flist[i])
    return(newflist)

#指定インデックスのファイルだけ削除
def kill_indfile(flist, gatchlist):
    newflist = []
    for i in range(len(flist)):
        if i in gatchlist:
            None
        else:
            newflist.append(flist[i])
    return(newflist)

def merge_option_csv(dirname): #マージ
    pdf_file_merger = PdfFileMerger()
    
    save = 'merge.pdf' #初期値
    #入力欄
    #coution allmerge=Trueの時はエラーになることがあった。
    allmerge = True #フォルダないすべてのファイルをマージする場合"True",そうでないとき"False"
    
    
    dirname = "dir" #ディレクトリを指定する場合に使用
    
    if dirname == "./": #dirname指定の場合は保存pdfのファイル名を自動設定
        None
    else:
        save = dirname+'.pdf'

    flist = dir_filelist(".pdf", dirname=dirname) #同じディレクトリ内にあるすべての.pdfファイルのリスト（dirnameをして
    print(flist)
    
    if allmerge:
        None
    else:
        gatchlist = csv_to_merge(flist, csv_in = csvin) #csvファイル指定の番号とファイルインデックス番号の対応
        flist = kill_unindfile(flist, gatchlist) #ファイルインデックス番号にないファイル除去

    for f in flist:
        pdf_file_merger.append(f) #ファイルのマージ操作
        print("A")
    
    pdf_file_merger.write(save) 
    print("B")
    pdf_file_merger.close()
    
dirname = "./" #マージするPDFの置場
csvin = "./toc.csv" #オプション。デフォルトでtoc.csv。allmerge=Falseの時使用。
merge_option_csv(dirname)

# PDFを画像として読込し、テキストデータを抽出（高精度なGoogle Keepで代用推奨）

In [ ]:
# ocr_card.py
import os
from PIL import Image
import pyocr
import pyocr.builders

# 1.インストール済みのTesseractのパスを通す
path_tesseract = "C:\\Program Files (x86)\\Tesseract-OCR"
if path_tesseract not in os.environ["PATH"].split(os.pathsep):
    os.environ["PATH"] += os.pathsep + path_tesseract

# 2.OCRエンジンの取得
tools = pyocr.get_available_tools()
tool = tools[0]

# 3.原稿画像の読み込み
#img_org = Image.open("./data/card_image/zairyucard_omote.jpg")
img_org = Image.open("./data/card_image/img1.jpg")

# 4.ＯＣＲ実行
builder = pyocr.builders.TextBuilder()
result = tool.image_to_string(img_org, lang="jpn", builder=builder)

print(result)

# PDF栞を更新する（CSV-PDF, PDF-CSV, PDF-PDF）

In [ ]:
import os,sys
from tkinter import *
from tkinter import ttk
from tkinter import filedialog

"""
01.PDFから栞のリストを取出す 220401
02.取出したリスト通りに栞を挿入しなおす。220401
03.PDFからPDFへ01をした後02を実行。221110
以下参考サイト
"""
#Tkinterに表示したタブの中に、さらにタブを作成したい。
#https://teratail.com/questions/rxqmt75cfd3lme

#Tkinterの階層構造とフレーム（Frame）
#https://denno-sekai.com/tkinter-frame/

#NOTE t.koba(2021.3.9)
#https://note.com/uranus_xii_jp/n/nbf8c42c0625b

#以下PDFの栞の書き込み
#https://jablogs.com/detail/49869

import fitz
import csv
from PyPDF2 import PdfFileWriter, PdfFileReader

def pdf_to_csv(pdf_in,csv_out):
    doc = fitz.open(pdf_in)
    page_count = doc.page_count # 総ページ数
    print('Page count: ', page_count)
    toc_list = doc.get_toc()

    with open(csv_out, 'w',newline="") as f: #空白行をなくすためnewline追加(220329)
        writer = csv.writer(f)
        writer.writerows(toc_list) #org
    f.close()
    return()

def csv_to_pdf(pdf_in, csv_out):
    with open(csv_out) as f: #csvの読込み
        reader = csv.reader(f, delimiter=',')
        l = [row for row in reader]
    
    l2 = [] #csvの空データ行削除
    for i in range(len(l)):
        if l[i] == []:
            pass
        else:
            #print(l[i])
            l2.append(l[i])

    output = PdfFileWriter() # open output
    input1 = PdfFileReader(open(pdf_in, 'rb')) # open input
    
    n = input1.getNumPages()
    for i in range(n):
        output.addPage(input1.getPage(i)) # insert page

    par_list = [] #階層のある場合のための階層親リスト
    kaiso_list = [] #階層番号のリスト

    for i in l2:
        kaiso = int(i[0])
        kaiso_list.append(kaiso)
        name = i[1]
        page = int(i[2])
        #print(kaiso,name,page)
        #page は -1 しないとずれる(2022.03.28)
        if kaiso == 1:
            par_list.append(output.addBookmark(name, page-1, parent=None))
        else:
            ln = len(kaiso_list)
            par_kaiso = kaiso-1 #親の階層
            r_kaiso = kaiso_list[::-1] #階層リストの逆順
            ind = r_kaiso.index(par_kaiso) #逆から親の階層検索
            p_ind = ln - ind - 1 #par_listの親階層インデクス
            par = par_list[p_ind]
            par_list.append(output.addBookmark(name, page-1, parent=par))

    base_pdf_in = os.path.splitext(os.path.basename(pdf_in))[0] #拡張子なしのファイル名
    sname = base_pdf_in + "_栞.pdf"
    #outputStream = open('result.pdf','wb') #creating result pdf JCT
    outputStream = open(sname,'wb') #221105
    output.write(outputStream) #writing to result pdf JCT
    outputStream.close() #closing result JCT

#221108開発 PDF[0]の栞をPDF[1]に適用する。
def pdf_to_pdf(pdf_in0, pdf_in1):
    csv_in0 = os.path.splitext(os.path.basename(pdf_in0))[0] + ".csv" #拡張子なしのファイル名
    pdf_to_csv(pdf_in0,csv_in0)
    print(pdf_in0)
    csv_to_pdf(pdf_in1,csv_in0)
    print("完了 pdf[0]->pdf[1]")

pdf_in = "non_topic.pdf"
csv_siori = "siori_data.csv"
csv_to_pdf(pdf_in, csv_siori)

# PDF栞を更新する（手入力Aパターン）

In [ ]:
def csv_to_pdf_from_list(pdf_in, l):
    
    l2 = [] #csvの空データ行削除
    for i in range(len(l)):
        if l[i] == []:
            pass
        else:
            l2.append(l[i])

    output = PdfFileWriter() # open output
    input1 = PdfFileReader(open(pdf_in, 'rb')) # open input
    
    n = input1.getNumPages()
    for i in range(n):
        output.addPage(input1.getPage(i)) # insert page

    par_list = [] #階層のある場合のための階層親リスト
    kaiso_list = [] #階層番号のリスト

    for i in l2:
        kaiso = int(i[0])
        kaiso_list.append(kaiso)
        name = i[1]
        page = int(i[2])
        if kaiso == 1:
            par_list.append(output.addBookmark(name, page-1, parent=None))
        else:
            ln = len(kaiso_list)
            par_kaiso = kaiso-1 #親の階層
            r_kaiso = kaiso_list[::-1] #階層リストの逆順
            ind = r_kaiso.index(par_kaiso) #逆から親の階層検索
            p_ind = ln - ind - 1 #par_listの親階層インデクス
            par = par_list[p_ind]
            par_list.append(output.addBookmark(name, page-1, parent=par))

    base_pdf_in = os.path.splitext(os.path.basename(pdf_in))[0] #拡張子なしのファイル名
    sname = base_pdf_in + "_栞.pdf"
    outputStream = open(sname,'wb') #221105
    output.write(outputStream) #writing to result pdf JCT
    outputStream.close() #closing result JCT

#更新栞の手入力Aパターン
pdf_in = "ogirinal.pdf"
#入力欄
toc_list = []
toc_list.append([1,"test1",1])
toc_list.append([1,"test2",2])

csv_to_pdf_from_list(pdf_in, toc_list)

# PDF栞を更新する（手入力Bパターン）

In [ ]:
def csv_to_pdf_from_list(pdf_in, l):
    
    l2 = [] #csvの空データ行削除
    for i in range(len(l)):
        if l[i] == []:
            pass
        else:
            l2.append(l[i])

    output = PdfFileWriter() # open output
    input1 = PdfFileReader(open(pdf_in, 'rb')) # open input
    
    n = input1.getNumPages()
    for i in range(n):
        output.addPage(input1.getPage(i)) # insert page

    par_list = [] #階層のある場合のための階層親リスト
    kaiso_list = [] #階層番号のリスト

    for i in l2:
        kaiso = int(i[0])
        kaiso_list.append(kaiso)
        name = i[1]
        page = int(i[2])
        if kaiso == 1:
            par_list.append(output.addBookmark(name, page-1, parent=None))
        else:
            ln = len(kaiso_list)
            par_kaiso = kaiso-1 #親の階層
            r_kaiso = kaiso_list[::-1] #階層リストの逆順
            ind = r_kaiso.index(par_kaiso) #逆から親の階層検索
            p_ind = ln - ind - 1 #par_listの親階層インデクス
            par = par_list[p_ind]
            par_list.append(output.addBookmark(name, page-1, parent=par))

    base_pdf_in = os.path.splitext(os.path.basename(pdf_in))[0] #拡張子なしのファイル名
    sname = base_pdf_in + "_栞.pdf"
    outputStream = open(sname,'wb') #221105
    output.write(outputStream) #writing to result pdf JCT
    outputStream.close() #closing result JCT

#更新栞の手入力(Bパターン)
pdf_in = "original.pdf"
#入力欄
toc_str = (
"""
1,test1,1
1,test2,2
1,test3,2
1,test4,2
"""
)

tl1 = [] #文字列データを栞リスト形式に変換
toc_list = toc_str.split("\n")
for i in range(len(toc_list)):
    if toc_list[i] == "":
        None
    else:
        tli = toc_list[i].split(",")
        tl1.append([int(tli[0]), str(tli[1]), int(tli[2])])

csv_to_pdf_from_list(pdf_in, tl1) #栞挿入

# EXCELをPDF印刷（１ファイルのシート1）

In [ ]:
import os
import win32com.client
excel = win32com.client.Dispatch("Excel.Application")
current_dir = %pwd #jupyter-labでカレントディレクトリ取得

def excel_to_pdf(current_dir, exc_in):
    wb = excel.Workbooks.Open(current_dir + "/" + exc_in + ".xlsx") #絶対パスでないとエラー
    wb.WorkSheets(1).Select()
    wb.ActiveSheet.ExportAsFixedFormat(0, current_dir + '/' + exc_in + '.pdf')
    wb.Close()

exc_in = r"\D18" #ファイル名
excel_to_pdf(current_dir, exc_in)

# EXCELをPDF印刷（１ファイルの全シート）

In [ ]:
import os
import win32com.client
excel = win32com.client.Dispatch("Excel.Application")
current_dir = %pwd #jupyter-labでカレントディレクトリ取得

def excel_to_pdf(current_dir, exc_in):
    wb = excel.Workbooks.Open(current_dir + "/" + exc_in + ".xlsx") #絶対パスでないとエラー
    count = 0
    for ws in wb.Sheets:
        ws.ExportAsFixedFormat(0,current_dir + "/" + exc_in + '-%i.pdf'%(count+1))
        count += 1
    wb.Close()

exc_in = r"\D18" #ファイル名
excel_to_pdf(current_dir, exc_in)

# EXCELをPDF印刷（複数ファイルのシート1）

In [ ]:
import os
import win32com.client
excel = win32com.client.Dispatch("Excel.Application")

def excel_to_pdf_multi(path):
    inputDir = path + r'\Excel' #元ファイルのあるフォルダ
    outputDir = path + r'\PDF' #出力ファイルのあるフォルダ
    for f in os.listdir(inputDir):
        base, ext = os.path.splitext(f)
        if ext == '.xlsx' and '~$' not in base:
            wb = excel.Workbooks.Open(os.path.join(inputDir,f))
            wb.WorkSheets(1).Select() #シートが複数ある場合は
            wb.ActiveSheet.ExportAsFixedFormat(0, outputDir + '/' + base + '.pdf')
            wb.Close()

current_dir = %pwd #jupyter-labでカレントディレクトリ取得
excel_to_pdf_multi(current_dir) #入力するのは作業フォルダの絶対パス

# EXCELをPDF印刷（複数ファイルの全シート）

In [ ]:
import os
import win32com.client
excel = win32com.client.Dispatch("Excel.Application")

def excel_to_pdf_multi2(path):
    inputDir = path + r'\Excel' #元ファイルのあるフォルダ
    outputDir = path + r'\PDF' #出力ファイルのあるフォルダ
    for f in os.listdir(inputDir):
        base, ext = os.path.splitext(f)
        if ext == '.xlsx' and '~$' not in base:
            wb = excel.Workbooks.Open(os.path.join(inputDir,f))
            count = 0
            for ws in wb.Sheets:
                ws.ExportAsFixedFormat(0, outputDir + '/' + base + '-%i.pdf'%(count+1))
                count += 1
            wb.Close()

current_dir = %pwd #jupyter-labでカレントディレクトリ取得
excel_to_pdf_multi2(current_dir) #入力するのは作業フォルダの絶対パス